## Imports

In [1]:
import h5py

from ssd import SSD300

Using TensorFlow backend.


## Constants

In [2]:
DATA_PATH = 'data.h5'

## Load data

In [3]:
with h5py.File(DATA_PATH, 'r') as f:
    anchors = f['main']['anchors'][:]
    data = f['main']['image'][:]
    bounding_boxes = f['main']['bounding_box'][:]
    classes = f['main']['class'][:]

KeyError: "Unable to open object (object 'anchors' doesn't exist)"

## Load model

In [ ]:
model = SSD300()

## Train model

In [ ]:
model.fit(data, y={'confidence': classes, 'localization': bounding_boxes})